# Feature Extraction using SIFT

In [2]:
import cv2
import pandas as pd
import numpy as np
import os
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

In [8]:
input0 = './Dataset/Augmented/'
temp = ['Bowlegs', 'Knock-knee', 'Normal Knee']

In [5]:
for i in temp:
    count = 0
    for filename in os.listdir(input0 + i):
        img = cv2.imread(input0 + i + '/' + filename)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        #initialise sift descriptor
        sift = cv2.SIFT_create()
        keypoints, descriptors = sift.detectAndCompute(gray, None)

        sift_image = cv2.drawKeypoints(gray, keypoints, img)
        
        #convert the descriptor array into a dataframe format
        out=pd.DataFrame(descriptors)
        
        #append to the csv file
        
        csv_data=out.to_csv('./SIFT/SIFT_' + i + '.csv', mode='a', index=False)
        count += 1
    print(i + ": " + str(count))

Bowlegs: 961
Knock-knee: 1944
Normal Knee: 2902


# K-means

In [4]:
data1 = pd.read_csv('C:\\Users\\sumit\\OneDrive\\Desktop\\FinalData.csv', dtype='uint8')
data2 = pd.read_csv('./SIFT/SIFT_Knock-knee.csv', dtype='uint8')
data3 = pd.read_csv('./SIFT/SIFT_Normal Knee.csv', dtype='uint8')


data1 = data1.astype('uint8')
data2 = data2.astype('uint8')
data3 = data3.astype('uint8')

In [5]:
#performing kmeans on each class
#Bowlegs
kmeans1 = KMeans(n_clusters=2)
kmeans1.fit(data1)

"""#Knock Knees
kmeans2 = KMeans(n_clusters=5)
kmeans2.fit(data2)

#Normal
kmeans3 = KMeans(n_clusters=5)
kmeans3.fit(data3)"""

'#Knock Knees\nkmeans2 = KMeans(n_clusters=5)\nkmeans2.fit(data2)\n\n#Normal\nkmeans3 = KMeans(n_clusters=5)\nkmeans3.fit(data3)'

In [6]:
hist1=np.histogram(kmeans1.labels_,bins=[0,1,2,3,4,5])
"""hist2=np.histogram(kmeans2.labels_,bins=[0,1,2,3,4,5])
hist3=np.histogram(kmeans3.labels_,bins=[0,1,2,3,4,5])"""

print('histogram of bowed')
print(hist1,"\n")

"""print('histogram of knocked')
print(hist2,"\n")

print('histogram of normal')
print(hist3,"\n")"""

histogram of bowed
(array([114625,  88210,      0,      0,      0], dtype=int64), array([0, 1, 2, 3, 4, 5])) 



'print(\'histogram of knocked\')\nprint(hist2,"\n")\n\nprint(\'histogram of normal\')\nprint(hist3,"\n")'

In [ ]:
#performing kmeans prediction of the entire apple dataset with the pretrained kmeans model
#initialising i=0; as its the first class
i=0
for j in temp:
    data=[]
    for filename in os.listdir(input0 + j):
        path = input0 + j + '/' + filename
        a=cv2.imread(path)
        gray = cv2.cvtColor(a, cv2.COLOR_BGR2GRAY)
        
        #initialise sift descriptor
        sift = cv2.SIFT_create()
        keypoints, descriptors = sift.detectAndCompute(gray, None)
        
        #convert the descriptor array into a dataframe format
        out=pd.DataFrame(descriptors)
        
        #predict values of feature vector with pretrained kmeans
        #ValueError: Buffer dtype mismatch, expected 'float' but got 'double', in order to avoid this dtype=np.double
        array_double = np.array(out, dtype=np.double)

        a=kmeans1.predict(array_double)
        hist=np.histogram(a,bins=[0,1,2,3,4,5])
        #append the dataframe into the array in append mode, the array will only have 5 values which will store the values in a row
        data.append(hist[0])
    
    #convert Array to Dataframe and append to the list
    Output = pd.DataFrame(data)
    #add row class 
    Output["Class"] = i 
    csv_data=Output.to_csv('./SIFT/SIFTH_'+ j + 'Final.csv', mode='a', index=False)
    i += 1

In [12]:
final = pd.read_csv('./SIFT/SIFT_BowlegsFinal.csv')
temp1 = final.append(pd.read_csv('./SIFT/SIFT_Knock-kneeFinal.csv'))
tc = pd.read_csv('./SIFT/SIFT_Normal KneeFinal.csv')
tcc = temp1.append(tc)
csv_data = tcc.to_csv('./SIFT/SIFTH_FeatureFinal.csv', mode='a', index=False)
print(tcc)

      0    1  2  3  4  Class
0    57   90  0  0  0      0
1    49  110  0  0  0      0
2    47  106  0  0  0      0
3    12    3  0  0  0      0
4    58   46  0  0  0      0
..   ..  ... .. .. ..    ...
953   5    4  0  0  0      2
954  45  175  0  0  0      2
955  50   21  0  0  0      2
956  39   13  0  0  0      2
957  48   16  0  0  0      2

[2902 rows x 6 columns]


C:\Users\sumit\AppData\Local\Temp\ipykernel_10636\3256986680.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp1 = final.append(pd.read_csv('./SIFT/SIFT_Knock-kneeFinal.csv'))
C:\Users\sumit\AppData\Local\Temp\ipykernel_10636\3256986680.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tcc = temp1.append(tc)


# Data spliting

In [13]:
df = pd.read_csv('./SIFT/SIFTH_FeatureFinal.csv')
X_train = df.iloc[:, 0:5]
Y_train = df.iloc[:, 5:6]
train_x, valid_x, train_y, valid_y = train_test_split(X_train, Y_train, 
                                                      test_size=0.3, 
                                                      stratify=Y_train, 
                                                      random_state=8)

# KNN

In [41]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error

knn = KNeighborsClassifier(n_neighbors=15)
  
knn.fit(train_x, train_y)

c:\Users\sumit\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=15)

In [42]:
print('\nResults obtained for KNN')

# accuracy on training data
y_pred_t = knn.predict(train_x)
train_data_accuracy = accuracy_score(y_pred_t, train_y)

print('\nResults obtained on Training Data')
print('Accuracy on Train data : ', train_data_accuracy)
print("Precision: ", precision_score(train_y, y_pred_t, average='macro'))
print("Recall: ", recall_score(train_y, y_pred_t, average='macro'))
print("F-score: ", f1_score(train_y, y_pred_t, average='macro'))

# Accuracy on test data
y_pred_ts = knn.predict(valid_x)
test_data_accuracy = accuracy_score(y_pred_ts, valid_y)

print('\nResults obtained on Testing Data')
print('Accuracy on Test data : ', test_data_accuracy)
print("Precision: ", precision_score(valid_y, y_pred_ts, average='macro'))
print("Recall: ", recall_score(valid_y, y_pred_ts, average='macro'))
print("F-score: ", f1_score(valid_y, y_pred_ts, average='macro'))



Results obtained for KNN

Results obtained on Training Data
Accuracy on Train data :  0.5307730182176268
Precision:  0.5313210527222668
Recall:  0.5304664176955508
F-score:  0.5306382948518115

Results obtained on Testing Data
Accuracy on Test data :  0.45120551090700345
Precision:  0.45528899276567464
Recall:  0.45134573132454486
F-score:  0.452435424345428


# XgBoost

In [32]:
import xgboost as xgb

classifier = xgb.XGBClassifier(
    n_estimators=100,
    reg_lambda=1,
    gamma=0,
    max_depth=3
)

classifier.fit(train_x, train_y)

[18:33:25] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\Users\sumit\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\sumit\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sumit\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=3, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [33]:
print('\nResults obtained for XgBoost')

# accuracy on training data
y_pred_t = classifier.predict(train_x)
train_data_accuracy = accuracy_score(y_pred_t, train_y)

print('\nResults obtained on Training Data')
print('Accuracy on Train data : ', train_data_accuracy)
print("Precision: ", precision_score(train_y, y_pred_t, average='macro'))
print("Recall: ", recall_score(train_y, y_pred_t, average='macro'))
print("F-score: ", f1_score(train_y, y_pred_t, average='macro'))

# Accuracy on test data
y_pred_ts = classifier.predict(valid_x)
test_data_accuracy = accuracy_score(y_pred_ts, valid_y)

print('\nResults obtained on Testing Data')
print('Accuracy on Test data : ', test_data_accuracy)
print("Precision: ", precision_score(valid_y, y_pred_ts, average='macro'))
print("Recall: ", recall_score(valid_y, y_pred_ts, average='macro'))
print("F-score: ", f1_score(valid_y, y_pred_ts, average='macro'))


Results obtained for XgBoost

Results obtained on Training Data
Accuracy on Train data :  0.604628261939931
Precision:  0.6049470021307489
Recall:  0.6042148272425494
F-score:  0.6020343903273256

Results obtained on Testing Data
Accuracy on Test data :  0.4362801377726751
Precision:  0.43168300611458904
Recall:  0.4357501569365976
F-score:  0.4318509282811958


# SVM

In [34]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier

subclassifier = SVC(kernel='rbf')
classifier = OneVsOneClassifier(estimator=subclassifier)
classifier.fit(train_x, train_y)

c:\Users\sumit\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


OneVsOneClassifier(estimator=SVC())

In [35]:
print('\nResults obtained for SVM')

# accuracy on training data
y_pred_t = classifier.predict(train_x)
train_data_accuracy = accuracy_score(y_pred_t, train_y)

print('\nResults obtained on Training Data')
print('Accuracy on Train data : ', train_data_accuracy)
print("Precision: ", precision_score(train_y, y_pred_t, average='macro'))
print("Recall: ", recall_score(train_y, y_pred_t, average='macro'))
print("F-score: ", f1_score(train_y, y_pred_t, average='macro'))

# Accuracy on test data
y_pred_ts = classifier.predict(valid_x)
test_data_accuracy = accuracy_score(y_pred_ts, valid_y)

print('\nResults obtained on Testing Data')
print('Accuracy on Test data : ', test_data_accuracy)
print("Precision: ", precision_score(valid_y, y_pred_ts, average='macro'))
print("Recall: ", recall_score(valid_y, y_pred_ts, average='macro'))
print("F-score: ", f1_score(valid_y, y_pred_ts, average='macro'))


Results obtained for SVM

Results obtained on Training Data
Accuracy on Train data :  0.4775972427375677
Precision:  0.4781223718449381
Recall:  0.4772281599707328
F-score:  0.4772341604415442

Results obtained on Testing Data
Accuracy on Test data :  0.45120551090700345
Precision:  0.45399112397844044
Recall:  0.4509063088512241
F-score:  0.4519475480854858


# Naive Bayes

In [36]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(train_x, train_y)

c:\Users\sumit\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB()

In [37]:
print('\nResults obtained for Naive Bayes')

# accuracy on training data
y_pred_t = classifier.predict(train_x)
train_data_accuracy = accuracy_score(y_pred_t, train_y)

print('\nResults obtained on Training Data')
print('Accuracy on Train data : ', train_data_accuracy)
print("Precision: ", precision_score(train_y, y_pred_t, average='macro'))
print("Recall: ", recall_score(train_y, y_pred_t, average='macro'))
print("F-score: ", f1_score(train_y, y_pred_t, average='macro'))

# Accuracy on test data
y_pred_ts = classifier.predict(valid_x)
test_data_accuracy = accuracy_score(y_pred_ts, valid_y)

print('\nResults obtained on Testing Data')
print('Accuracy on Test data : ', test_data_accuracy)
print("Precision: ", precision_score(valid_y, y_pred_ts, average='macro'))
print("Recall: ", recall_score(valid_y, y_pred_ts, average='macro'))
print("F-score: ", f1_score(valid_y, y_pred_ts, average='macro'))


Results obtained for Naive Bayes

Results obtained on Training Data
Accuracy on Train data :  0.4155588380108321
Precision:  0.4426633400284592
Recall:  0.41283579166952483
F-score:  0.38350697126628047

Results obtained on Testing Data
Accuracy on Test data :  0.4087256027554535
Precision:  0.41049536905405715
Recall:  0.4062421531701193
F-score:  0.3735772005059195
